In [1]:
import os, ast, json, uuid
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "browser"
print(pio.renderers)

Renderers configuration
-----------------------
    Default renderer: 'browser'
    Available renderers:
        ['plotly_mimetype', 'jupyterlab', 'nteract', 'vscode',
         'notebook', 'notebook_connected', 'kaggle', 'azure', 'colab',
         'cocalc', 'databricks', 'json', 'png', 'jpeg', 'jpg', 'svg',
         'pdf', 'browser', 'firefox', 'chrome', 'chromium', 'iframe',
         'iframe_connected', 'sphinx_gallery', 'sphinx_gallery_png']



In [2]:
pd.set_option('display.max_columns', None)
sns.set_theme()

In [3]:
os.getcwd()

'c:\\users\\user\\root_dir\\src\\nct_distr'

In [7]:
distreport_redis_path = "C:/Users/user/Downloads/dist_report_redis-2025-07-31 - Sheet1.csv"

In [4]:
dist3_path = "C:/Users/user/root_dir/data/nct-distr/dist-report-v3.csv"

In [5]:
df = pd.read_csv(dist3_path)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 288522 entries, 0 to 288521
Data columns (total 25 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   Unnamed: 0                                288522 non-null  int64  
 1   ReportDisburseNarcoticBeYear              288522 non-null  float64
 2   ReportDisburseNarcoticMonth               288522 non-null  float64
 3   ProductTypeName                           288522 non-null  object 
 4   MonthlyReportNarcoticName                 288522 non-null  object 
 5   MonthlyReportNarcoticBalanceForwardValue  288522 non-null  float64
 6   MonthlyReportNarcoticReceiveValue         288522 non-null  float64
 7   MonthlyReportNarcoticPaidValue            288522 non-null  float64
 8   MonthlyReportNarcoticRemainValue          288522 non-null  float64
 9   ReferCustomerName                         288522 non-null  object 
 10  CustomerAddressProvi

In [9]:
df_og = pd.read_csv(distreport_redis_path)

In [10]:
df_og.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395588 entries, 0 to 395587
Data columns (total 17 columns):
 #   Column                                    Non-Null Count   Dtype  
---  ------                                    --------------   -----  
 0   ReportDisburseNarcoticBeYear              395588 non-null  int64  
 1   ReportDisburseNarcoticMonth               395588 non-null  int64  
 2   ProductTypeName                           395588 non-null  object 
 3   MonthlyReportNarcoticName                 395588 non-null  object 
 4   MonthlyReportNarcoticBalanceForwardValue  395588 non-null  float64
 5   MonthlyReportNarcoticReceiveValue         395588 non-null  float64
 6   MonthlyReportNarcoticPaidValue            395588 non-null  float64
 7   MonthlyReportNarcoticRemainValue          395588 non-null  float64
 8   ReferCustomerName                         395588 non-null  object 
 9   CustomerAddressProvinceName               395588 non-null  object 
 10  CustomerAddress     

In [11]:
df_og['ProductTypeName'].value_counts()

ProductTypeName
(วจ. 2) วัตถุออกฤทธิ์ในประเภท 2     213340
(ยส. 2) ยาเสพติดให้โทษในประเภท 2    182248
Name: count, dtype: int64

In [17]:
df_og['CustomerTypeName'].value_counts().reset_index().sort_values(by="count", ascending=False)

,CustomerTypeName,count
0,สถานพยาบาลของรัฐ,186143
1,โรงพยาบาลเอกชน,114597
2,คลินิก,78178
3,สถานพยาบาลสัตว์,15740
4,สถานพยาบาลทันตกรรม,489
5,ขนส่งระหว่างประเทศ,158
6,ยานพาหนะที่ใช้ในการขนส่งสาธารณะระหว่างประเทศ,70
7,ประเภท ข เดินเรือ,68
8,บริษัท วงศ์สมุทรมาริไทม์ จำกัด,50
9,ใบอนุญาตมีไว้ครอบครองประเภท 2 หมวด ข (เรือกลเ...,36


In [18]:
df_og['ReportDisburseNarcoticRecordType'].value_counts()

ReportDisburseNarcoticRecordType
RECORD    199505
ONLINE    196083
Name: count, dtype: int64